# Task 6 : Fault Detector

## Submitted By: Daniya

### Implementation: 
            This system accept Rectangular shape of blue color products only anyother shape od blue color product is consider as defective

In [2]:
import cv2
import numpy as np
from win32com.client import Dispatch
import winsound

In [7]:
def empty(img):
    pass

def alert(str):
    speak= Dispatch(("SAPI.SpVoice"))
    speak.Speak(str)

# Resize the video Frame
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)     

# Getting Upper and Lower Bounds
I have used track bar to get the upper and lowers bounds on blue colors

    cv2.namedWindow("TrackBar")
    cv2.resizeWindow("TracKBar",600,300)
    cv2.createTrackbar("hue_min","TrackBar",0,179,empty)
    cv2.createTrackbar("hue_max","TrackBar",179,179,empty)
    cv2.createTrackbar("sat_min","TrackBar",0,255,empty)
    cv2.createTrackbar("sat_max","TrackBar",255,255,empty)
    cv2.createTrackbar("val_min","TrackBar",0,255,empty)
    cv2.createTrackbar("val_max","TrackBar",255,255,empty)

   
    getting mask values
    hue_min= cv2.getTrackbarPos("hue_min","TrackBar")
    hue_max= cv2.getTrackbarPos("hue_max","TrackBar")
    sat_min= cv2.getTrackbarPos("sat_min","TrackBar")
    sat_max= cv2.getTrackbarPos("hue_min","TrackBar")
    val_min= cv2.getTrackbarPos("val_min","TrackBar")
    val_max= cv2.getTrackbarPos("val_max","TrackBar")
    

In [50]:
video = cv2.VideoCapture("test.mp4") #0

while(video.isOpened()):
    fault_products=False
    ret , img = video.read()
#     img= cv2.imread("imgtest.jfif")
#     cv2.imshow("img",img)
    if(ret):
        img = rescale_frame(img, percent=50)
        #making  mask
        hsv = cv2.cvtColor(img , cv2.COLOR_BGR2HSV)

        #Blue Color Object is Accepted
        lower= np.array([104,68,0])
        upper= np.array([134,255,241])

        mask = cv2.inRange(hsv,lower,upper)

        #FIND CONTOURS
        _,cnts,hei = cv2.findContours(mask,cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)
        for c in cnts:
            area= cv2.contourArea(c)
            approx=[1,2]
            x,y,w,h=0,0,0,0
            if area>300:
                peri=cv2.arcLength(c,True)
                approx=cv2.approxPolyDP(c,0.02*peri,True)
                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

            if len(approx)==2:
                pass
            elif len(approx)==4:
                cv2.putText(img,"Correct",(x,y+h+45) , cv2.FONT_HERSHEY_COMPLEX, 0.5 , (0,255,0),2)
            elif len(approx)==3:
                cv2.rectangle(img, (0, 10), (150, 50), (0, 255, 255), -1)
                cv2.putText(img,"ALERT",(5,40), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0),2) 
                cv2.putText(img,"DEFECTED",(x,y+h+25) , cv2.FONT_HERSHEY_COMPLEX, 0.5 , (0,0,255),2)
#                 winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
#                 alert("Defected Product Found")
            else:
                cv2.rectangle(img, (0, 10), (150, 50), (0, 255, 255), -1)
                cv2.putText(img,"ALERT",(5,40), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0),2) 
                cv2.putText(img,"DEFECTED",(x,y+h+25) , cv2.FONT_HERSHEY_COMPLEX, 0.5 , (0,0,255),2)      
#                 winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
#                 alert("Defected Product Found")
        cv2.imshow("Frame",img)
        k=cv2.waitKey(1)
        if k == ord('q'):
            break
    else:
        cv2.destroyAllWindows()
        break
        
video.release()
cv2.destroyAllWindows()